##### Copyright 2025 Google LLC.

In [39]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API: Self-critique prompt optimization

<a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/examples/prompting/Self_critique_prompt_optimization.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

Prompt engineering often involves manual trial and error. You write a prompt, evaluate the output, tweak the prompt, and repeat. This notebook demonstrates how to automate this process by having Gemini critique its own outputs and suggest prompt improvements.

This technique, sometimes called **meta-prompting** or **self-critique**, uses the model to:

1. Generate a response from an initial prompt
2. Critique the quality of that response
3. Identify specific weaknesses
4. Rewrite the prompt to address those weaknesses
5. Generate an improved response

By the end of this notebook, you will understand how to implement an iterative prompt optimization loop that can help you develop better prompts faster.

## Setup

### Install SDK

In [40]:
%pip install -U -q "google-genai>=1.0.0"


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Set up your API key

To run the following cell, your API key must be stored in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
from google import genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Select the model you want to use from the available options:

In [42]:
MODEL_ID = "gemini-2.5-flash"  # @param ["gemini-2.5-flash-lite", "gemini-2.5-flash", "gemini-2.5-pro", "gemini-2.5-flash-preview", "gemini-3-flash-preview", "gemini-3-pro-preview"] {"allow-input":true, isTemplate: true}

In [43]:
from IPython.display import Markdown, display

## The problem: weak prompts produce weak results

Consider a common scenario: you need the model to explain a technical concept, but your initial prompt is vague. The output might be generic, miss key details, or lack the structure you need.

Here's a deliberately weak prompt to demonstrate:

In [44]:
# Define the task context - this stays constant throughout optimization
TASK_DESCRIPTION = "Explain how neural networks learn"

# Initial weak prompt - vague and lacks specificity
initial_prompt = "Explain how neural networks learn."

### Generate the initial response

In [45]:
initial_response = client.models.generate_content(
    model=MODEL_ID,
    contents=initial_prompt
)

print("=" * 60)
print("INITIAL PROMPT:")
print("=" * 60)
print(initial_prompt)
print("\n" + "=" * 60)
print("INITIAL RESPONSE:")
print("=" * 60)
display(Markdown(initial_response.text))

INITIAL PROMPT:
Explain how neural networks learn.

INITIAL RESPONSE:


Neural networks learn through a process of **iterative refinement**, much like a student learns by trial and error, getting feedback, and adjusting their approach. The core idea is to adjust the internal parameters (weights and biases) of the network until it can accurately map input data to desired output.

Here's a breakdown of the key steps involved:

1.  **Initialization:**
    *   Before learning begins, the network's **weights** (the strength of connections between neurons) and **biases** (an additional value that determines how easily a neuron activates) are set to small, random values. Think of these as the network's initial, uneducated guesses.

2.  **Forward Pass (Making a Prediction):**
    *   **Input Data:** The network is fed a piece of input data (e.g., an image, a row of numbers, text).
    *   **Calculation:** This input data travels through the network, from the input layer, through any hidden layers, to the output layer.
        *   At each neuron, a calculation is performed: `(sum of inputs * weights) + bias`.
        *   This result is then passed through an **activation function** (e.g., ReLU, Sigmoid), which introduces non-linearity and determines if/how much the neuron "fires."
    *   **Output:** The network produces an output (a prediction) based on these calculations and its current weights and biases. This is its "initial guess."

3.  **Measuring the Error (Loss Function):**
    *   **Comparison:** The network's prediction is compared to the **actual correct answer** (the "ground truth" or "label") that was provided with the input data during training.
    *   **Loss Calculation:** A **loss function** (also called a cost function, e.g., Mean Squared Error for regression, Cross-Entropy for classification) quantifies how "wrong" the network's prediction was. A higher loss value means a larger error. The goal is to minimize this loss.

4.  **Backpropagation (Credit Assignment):**
    *   This is the crucial step where the network "learns" from its mistakes.
    *   **Gradient Calculation:** The error from the loss function is propagated backward through the network, from the output layer towards the input layer. During this process, the network calculates the **gradient** (the partial derivative of the loss with respect to each weight and bias).
    *   **Understanding Gradients:** The gradient tells us two things for each weight and bias:
        *   **Direction:** Which way should this parameter be adjusted (increased or decreased) to reduce the loss?
        *   **Magnitude:** How much should this parameter be adjusted, relative to others, to have the most impact on reducing the loss? (Parameters that contributed more to the error will have larger gradients).

5.  **Weight and Bias Update (Optimization):**
    *   **Gradient Descent:** An optimization algorithm, most commonly **Gradient Descent** (or its variants like Adam, RMSprop, etc.), uses the gradients calculated during backpropagation to adjust the weights and biases.
    *   **Adjustment Rule:** Each weight and bias is updated using the formula:
        `New Weight/Bias = Old Weight/Bias - (Learning Rate * Gradient)`
    *   **Learning Rate:** This is a crucial hyperparameter that controls the size of the steps taken during each update.
        *   A **high learning rate** can make the network learn quickly but risk overshooting the optimal values.
        *   A **low learning rate** can make learning slow but more stable.

6.  **Iteration (Epochs):**
    *   The entire process (forward pass, error calculation, backpropagation, parameter update) is repeated many, many times for all the training examples in the dataset.
    *   One complete pass through the entire training dataset is called an **epoch**.
    *   Over thousands or millions of iterations and many epochs, the weights and biases are gradually refined. The network gets "smarter" with each adjustment, leading to progressively smaller errors and more accurate predictions.

**In essence, neural networks learn by:**

*   Making an initial prediction.
*   Measuring how wrong that prediction was.
*   Figuring out which internal parameters (weights and biases) contributed most to the error.
*   Adjusting those parameters slightly in the direction that would reduce the error.
*   Repeating this cycle countless times until the error is minimized and the network can generalize well to new, unseen data.

## Step 1: Critique the output

Now, ask the model to critically evaluate its own response. The critique prompt should ask for specific, actionable feedback.

In [46]:
def critique_response(task, prompt, response_text):
    """
    Ask the model to critique a response and identify weaknesses.
    """
    critique_prompt = f"""
You are a prompt engineering expert. Analyze the following prompt and its 
response, then provide a detailed critique.

TASK: {task}

PROMPT USED:
{prompt}

RESPONSE GENERATED:
{response_text}

Provide your critique in this format:

STRENGTHS:
- List what the response did well

WEAKNESSES:
- List specific problems with the response
- Focus on: clarity, completeness, structure, accuracy, relevance

PROMPT ISSUES:
- Identify what was missing or unclear in the original prompt
- Explain how prompt weaknesses led to response weaknesses

QUALITY SCORE: [1-10]
"""
    
    critique = client.models.generate_content(
        model=MODEL_ID,
        contents=critique_prompt
    )
    return critique.text

In [48]:
critique_1 = critique_response(TASK_DESCRIPTION, initial_prompt, initial_response.text)

print("=" * 60)
print("CRITIQUE OF INITIAL RESPONSE:")
print("=" * 60)
display(Markdown(critique_1))

CRITIQUE OF INITIAL RESPONSE:


STRENGTHS:
-   **Exceptional Clarity and Accessibility:** The response uses clear, concise language, effectively breaking down a complex topic into understandable parts. Analogies (student learning, initial guesses) are well-chosen and aid comprehension. Technical terms are introduced and explained without being overwhelming.
-   **Outstanding Logical Structure:** The explanation follows a perfect chronological and logical flow, from initialization to iterative refinement. The use of numbered headings, bold text for key terms, and bullet points significantly enhances readability and makes the process easy to follow.
-   **Comprehensive Coverage:** It covers all essential steps of neural network learning: Initialization, Forward Pass, Loss Function, Backpropagation, Weight/Bias Update (Optimization), and Iteration (Epochs). No critical step is omitted for a general explanation.
-   **High Accuracy:** All technical concepts, including weights, biases, activation functions, loss functions, gradients, gradient descent, and learning rate, are accurately described. The explanation of gradients (direction and magnitude) is particularly well-articulated, simplifying a often-difficult concept.
-   **Effective Explanation of Complex Concepts:** Backpropagation and Gradient Descent, which are often stumbling blocks in understanding neural networks, are explained with remarkable clarity and precision, focusing on their purpose and mechanism without getting bogged down in excessive mathematical detail.
-   **Excellent Summarization:** The concluding "In essence" section provides a succinct and impactful recap, reinforcing the core learning mechanism.
-   **Directly Relevant:** The response fully and completely answers the prompt without extraneous information.

WEAKNESSES:
-   This response is exceptionally strong, and identifying true weaknesses that detract from its quality for the given prompt is challenging.
-   **Minor Opportunity for Deeper Dive (Not a flaw):** For a slightly more advanced audience, mentioning the concept of mini-batch gradient descent (more common in practice) rather than implying full-batch, or providing a conceptual visual analogy for the loss landscape, could enhance it further. However, for a general explanation, omitting these details keeps it focused and accessible, aligning well with what a broad prompt would typically require.

PROMPT ISSUES:
-   **Lack of Specificity:** The prompt "Explain how neural networks learn" is very open-ended. It does not specify:
    *   **Target Audience:** Is the explanation for a beginner, an intermediate learner, or an expert?
    *   **Desired Depth:** Should it be a high-level overview, a detailed technical explanation, or include mathematical formulations?
    *   **Length or Format Constraints:** Are there any preferred length limits or formatting requirements (e.g., step-by-step, analogy-based, summary only)?
-   **Impact on Response (Minimal in this case):** While a lack of specificity *can* often lead to responses that are either too simplistic or too complex for the user's actual need, in this particular instance, the model made excellent assumptions. It produced a comprehensive, well-structured, and accessible explanation that caters well to a general audience seeking a solid understanding of the topic. The model's "intelligence" in interpreting the implicit need for clarity and thoroughness compensated for the prompt's generic nature, resulting in an exceptionally high-quality response despite the prompt's weakness.

QUALITY SCORE: 10/10

## Step 2: Rewrite the prompt

Based on the critique, ask the model to generate an improved prompt that addresses the identified weaknesses.

In [49]:
def rewrite_prompt(task, original_prompt, critique):
    """
    Generate an improved prompt based on the critique.
    """
    rewrite_instruction = f"""
You are a prompt engineering expert. Based on the critique below, rewrite the 
prompt to address all identified weaknesses.

TASK: {task}

ORIGINAL PROMPT:
{original_prompt}

CRITIQUE:
{critique}

Write an improved prompt that:
1. Addresses all weaknesses mentioned in the critique
2. Is clear and specific about expectations
3. Includes relevant constraints (format, length, audience, etc.)
4. Guides the model toward a higher quality response

Return ONLY the improved prompt, nothing else. Do not include explanations 
or commentary.
"""
    
    result = client.models.generate_content(
        model=MODEL_ID,
        contents=rewrite_instruction
    )
    return result.text.strip()

In [50]:
improved_prompt_1 = rewrite_prompt(TASK_DESCRIPTION, initial_prompt, critique_1)

print("=" * 60)
print("IMPROVED PROMPT (Iteration 1):")
print("=" * 60)
print(improved_prompt_1)

IMPROVED PROMPT (Iteration 1):
Explain how neural networks learn.

**Audience:** An intermediate learner with basic programming knowledge but no prior machine learning experience.

**Depth:** Provide a detailed conceptual explanation of the learning process. Focus on the core mechanisms, their purpose, and how they contribute to learning, avoiding complex mathematical derivations. Accurately define all technical terms introduced.

**Content Requirements:**
*   Cover the essential steps: Initialization, Forward Pass, Loss Function, Backpropagation, Weight/Bias Update (Optimization), and Iteration (Epochs).
*   Briefly introduce mini-batch gradient descent as a common, practical optimization strategy.
*   Include a conceptual analogy for the loss landscape to aid understanding of gradient descent.

**Format and Style:**
*   Structure the explanation with clear, numbered headings for each step.
*   Use clear, concise language.
*   Employ well-chosen analogies to simplify complex ideas.
* 

## Step 3: Generate response with improved prompt

In [51]:
improved_response_1 = client.models.generate_content(
    model=MODEL_ID,
    contents=improved_prompt_1
)

print("=" * 60)
print("IMPROVED RESPONSE (Iteration 1):")
print("=" * 60)
display(Markdown(improved_response_1.text))

IMPROVED RESPONSE (Iteration 1):


Neural networks are incredibly powerful tools for tasks like image recognition, natural language processing, and medical diagnosis. At their core, they learn by trying to make sense of data, making predictions, seeing how wrong they are, and then adjusting themselves to be less wrong next time. This process, while seemingly complex, can be broken down into several understandable steps.

Imagine you're trying to teach a child to identify different animals. Initially, they might make wild guesses. You provide feedback ("That's a cat, not a dog!"), and over time, they learn to refine their understanding. Neural networks learn in a remarkably similar, albeit mathematical, fashion.

Here's a detailed conceptual explanation of how neural networks learn:

---

### The Learning Process: A Step-by-Step Guide

#### 1. Initialization: Starting with a Guess

Before a neural network can learn anything, it needs a starting point. This involves setting up its internal parameters: **weights** and **biases**.
*   **Weights:** These are numerical values that represent the strength of connection between neurons (the fundamental processing units) in different layers. A higher weight means that input has a stronger influence on the next neuron. Think of them as the "importance sliders" for different pieces of information.
*   **Biases:** These are additional numerical values attached to each neuron, acting like an adjustable threshold. They allow a neuron to activate even if all its inputs are zero, or to remain inactive even with some positive inputs, providing more flexibility. Think of them as "offset adjusters."

Initially, all weights and biases are set to small, random values. Why random? If they all started with the same values, every neuron in a layer would learn the exact same thing, making the network ineffective. Random initialization ensures each neuron starts with a unique perspective, allowing the network to learn diverse features. It's like giving different students slightly varied initial assumptions to encourage independent thought.

#### 2. Forward Pass: Making a Prediction

With the network initialized, we can now feed it some data. This is called a **forward pass**.
1.  **Input:** The raw data (e.g., pixel values of an image, words in a sentence) is fed into the network's **input layer**.
2.  **Processing through Layers:** This input then travels through one or more **hidden layers** and finally reaches the **output layer**. In each neuron within these layers:
    *   It receives numerical inputs from the neurons in the previous layer.
    *   Each input is multiplied by its corresponding **weight**.
    *   All these weighted inputs are summed together, and the neuron's **bias** is added to this sum.
    *   This total sum then passes through an **activation function**. This function introduces non-linearity, allowing the network to learn complex patterns (e.g., a "step" function decides if a neuron "fires" or not, or a smooth curve outputs a probability).
    *   The output of the activation function becomes the input for the neurons in the next layer.
3.  **Prediction:** The values generated by the **output layer** represent the network's **prediction** for the given input (e.g., a probability that the image is a cat, or a predicted stock price).

This entire process is like a series of interconnected calculators, each performing a simple calculation and passing its result to the next in line, ultimately arriving at a final answer.

#### 3. Loss Function: Measuring the Error

After the network makes a prediction, we need to know how good or bad that prediction is. This is where the **loss function** (or cost function) comes in.
*   The loss function is a mathematical formula that quantifies the difference between the network's **prediction** and the true, correct answer (known as the **ground truth**).
*   For example, if the network predicts "dog" but the image is actually a "cat," the loss function would output a high value, indicating a large error. If it predicts "cat" for a "cat," the loss would be very low, ideally zero.
*   Different tasks use different loss functions. For predicting a continuous value (like house price), **Mean Squared Error** is common. For classification (like identifying an animal), **Cross-Entropy** is often used.

The goal of learning is always to **minimize this loss**. Think of it as a golf score – the lower the score, the better the performance.

#### 4. Backpropagation: Tracing the Blame

Now that we know *how wrong* the network's prediction was (from the loss function), the crucial next step is to figure out *who* (which weights and biases) is responsible for that error, and by how much. This is the job of **backpropagation**.

Backpropagation is an algorithm that efficiently calculates the **gradient** of the loss function with respect to every single weight and bias in the network.
*   The **gradient** tells us two things: the *direction* in which to adjust a weight/bias to reduce the loss, and the *magnitude* of that adjustment (how much impact that weight/bias had on the error).
*   Conceptually, backpropagation works by propagating the error backward from the output layer through the hidden layers, all the way to the input layer. It's like a blame assignment process:
    *   It starts by calculating how much each output neuron contributed to the total loss.
    *   Then, it moves backward, determining how much each weight and bias in the *previous* layer contributed to the error propagated from the layer above.
    *   This process continues layer by layer, meticulously calculating each parameter's individual responsibility for the overall error.

Imagine a complex assembly line where a defective product comes out at the end. Backpropagation is like tracing back through each workstation and component, figuring out which specific part introduced the defect and how significantly it contributed to the final flaw. It provides a precise map of how to change each parameter to improve the outcome.

#### 5. Weight and Bias Update (Optimization): Adjusting the Knobs

With the gradients calculated via backpropagation, we now know exactly how to adjust each weight and bias to reduce the loss. This adjustment is handled by an **optimization algorithm**, the most fundamental of which is **Gradient Descent**.

*   **Gradient Descent:** The core idea is simple: we want to move down the "hill" of the loss landscape to find the lowest point (minimal loss). Since the gradient points in the direction of the steepest *increase* in loss, we adjust each weight and bias in the *opposite* direction of its gradient.
    *   `New Weight = Old Weight - (Learning Rate * Gradient of Weight)`
    *   `New Bias = Old Bias - (Learning Rate * Gradient of Bias)`

*   **Learning Rate:** This is a crucial **hyperparameter** (a setting that is not learned by the network but set by the programmer). It determines the size of the steps we take down the loss landscape.
    *   A large learning rate might cause us to overshoot the minimum or even diverge.
    *   A small learning rate might make the learning process very slow.
    *   Finding the right learning rate is often key to effective training.

**Conceptual Analogy for the Loss Landscape:**
Imagine you are blindfolded on a vast, uneven landscape (the **loss landscape**). Your goal is to find the lowest point in this landscape (where the loss is minimized). You can't see, but you can feel the slope under your feet. Gradient descent is like taking a small step in the direction that feels steepest *downhill*. You repeat this process, always stepping downhill, until you reach a valley or a flat area (a local or global minimum). The learning rate determines how big each step you take is.

#### 6. Iteration (Epochs): Repeating the Cycle

A single forward pass, loss calculation, backpropagation, and parameter update is just one small step in the learning process. Neural networks rarely learn effectively from just one go.
*   The entire dataset is typically passed through the network multiple times. Each full pass over the entire training dataset is called an **epoch**.
*   We repeat the entire learning cycle (steps 2 through 5) for many epochs. With each epoch, the weights and biases are slightly refined, leading to incrementally better predictions and lower loss.

This iterative process is similar to a student studying for an exam. They don't just read the textbook once and ace it. They review material, test themselves, identify weak areas, restudy those specific topics, and repeat until they've mastered the subject.

#### 7. Mini-batch Gradient Descent: Learning in Chunks

While Gradient Descent sounds straightforward, processing the entire training dataset for every single weight update (full batch gradient descent) can be computationally expensive and slow for very large datasets.
*   To address this, **Mini-batch Gradient Descent** is commonly used. Instead of processing the entire dataset at once, the data is divided into smaller, randomly sampled groups called **mini-batches**.
*   The network performs a forward pass, calculates loss, and updates weights/biases for one mini-batch at a time. Once all mini-batches have been processed, one epoch is complete.
*   This approach offers a practical balance: it's faster than full batch gradient descent and provides a more stable update direction than **Stochastic Gradient Descent (SGD)**, which uses only one data point at a time. It also efficiently utilizes modern computing hardware.

---

### Summary

In essence, a neural network learns through a continuous cycle of **prediction, error measurement, and adjustment**. It starts with random guesses (initialized weights/biases), makes a prediction (forward pass), compares that prediction to the truth (loss function), precisely calculates how each internal parameter contributed to the error (backpropagation), and then slightly tweaks those parameters to reduce future error (gradient descent with a learning rate). This entire process is repeated over many iterations (epochs) using small chunks of data (mini-batches) until the network becomes proficient at its task, effectively learning to identify complex patterns and make accurate predictions.

## Iteration 2: Further refinement

Run the critique-rewrite cycle again to see if additional improvements are possible.

In [52]:
critique_2 = critique_response(TASK_DESCRIPTION, improved_prompt_1, improved_response_1.text)

print("=" * 60)
print("CRITIQUE (Iteration 2):")
print("=" * 60)
display(Markdown(critique_2))

CRITIQUE (Iteration 2):


Here's a detailed critique of the prompt and its response:

---

### STRENGTHS:
*   **Exceptional Adherence to Content Requirements:** The response meticulously covers all requested essential steps: Initialization, Forward Pass, Loss Function, Backpropagation, Weight/Bias Update (Optimization), and Iteration (Epochs). It also successfully introduces mini-batch gradient descent as requested.
*   **Outstanding Conceptual Clarity and Accuracy:** The explanation of each step is conceptually sound, accurate, and avoids unnecessary mathematical complexity, perfectly aligning with the "detailed conceptual explanation" and "avoiding complex mathematical derivations" requirements.
*   **Highly Accessible Language for the Audience:** The language used is consistently clear, concise, and perfectly pitched for an "intermediate learner with basic programming knowledge but no prior machine learning experience." Technical jargon is handled well.
*   **Excellent Use of Analogies:** The response excels in using well-chosen and effective analogies (e.g., teaching a child, importance sliders, interconnected calculators, golf score, blame assignment, blindfolded on a loss landscape, student studying for an exam) to simplify complex ideas and enhance understanding, making the abstract concrete.
*   **Thorough Technical Term Definition:** All key technical terms are accurately defined upon introduction, further aiding the target audience's comprehension. Bold text for these terms is consistently applied.
*   **Strong Structure and Flow:** The explanation flows logically from one step to the next, building understanding progressively. The introductory and concluding summaries effectively frame the core learning mechanism.
*   **Precise Formatting:** The response correctly uses clear, *numbered* headings for each step (`#### 1. Initialization: Starting with a Guess`), which was a specific prompt requirement, demonstrating careful attention to detail.
*   **Comprehensive Coverage:** The response provides a truly comprehensive overview of the learning process, fulfilling the prompt's request for a "comprehensive, accurate, and highly accessible" explanation.

### WEAKNESSES:
*   **Slight Word Count Exceedance:** The prompt specified an approximate length of "800-1200 words." The generated response is approximately 1319 words, exceeding the upper bound by about 10%. While not drastic, it is an area where conciseness could be slightly improved to fit within the requested range without sacrificing essential detail.

### PROMPT ISSUES:
*   The original prompt is exceptionally well-crafted. It is clear, highly detailed, and comprehensive in its requirements for audience, depth, content, and formatting. There are no identifiable weaknesses or ambiguities in the prompt that led to any issues in the response. The minor word count overage in the response is more attributable to the model's execution of providing a "detailed conceptual explanation" rather than a flaw in the prompt's clarity regarding the approximate word count.

### QUALITY SCORE: 9.5/10

In [53]:
improved_prompt_2 = rewrite_prompt(TASK_DESCRIPTION, improved_prompt_1, critique_2)

print("=" * 60)
print("IMPROVED PROMPT (Iteration 2):")
print("=" * 60)
print(improved_prompt_2)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 24.914550338s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'},

In [ ]:
improved_response_2 = client.models.generate_content(
    model=MODEL_ID,
    contents=improved_prompt_2
)

print("=" * 60)
print("IMPROVED RESPONSE (Iteration 2):")
print("=" * 60)
display(Markdown(improved_response_2.text))

## Iteration 3: Final refinement

One more iteration to maximize prompt quality.

In [ ]:
critique_3 = critique_response(TASK_DESCRIPTION, improved_prompt_2, improved_response_2.text)

print("=" * 60)
print("CRITIQUE (Iteration 3):")
print("=" * 60)
display(Markdown(critique_3))

In [ ]:
improved_prompt_3 = rewrite_prompt(TASK_DESCRIPTION, improved_prompt_2, critique_3)

print("=" * 60)
print("FINAL OPTIMIZED PROMPT (Iteration 3):")
print("=" * 60)
print(improved_prompt_3)

In [ ]:
final_response = client.models.generate_content(
    model=MODEL_ID,
    contents=improved_prompt_3
)

print("=" * 60)
print("FINAL RESPONSE (Iteration 3):")
print("=" * 60)
display(Markdown(final_response.text))

## Compare: Before and after

Let's compare the prompt evolution and have the model evaluate the improvement.

In [ ]:
print("=" * 60)
print("PROMPT EVOLUTION")
print("=" * 60)

print("\n[ORIGINAL PROMPT]")
print(initial_prompt)

print("\n" + "-" * 40)
print("\n[ITERATION 1]")
print(improved_prompt_1)

print("\n" + "-" * 40)
print("\n[ITERATION 2]")
print(improved_prompt_2)

print("\n" + "-" * 40)
print("\n[FINAL PROMPT]")
print(improved_prompt_3)

In [ ]:
comparison_prompt = f"""
Compare these two responses to the task: "{TASK_DESCRIPTION}"

RESPONSE A (from weak prompt):
{initial_response.text[:2000]}...

RESPONSE B (from optimized prompt):
{final_response.text[:2000]}...

Provide a brief comparison:
1. What specific improvements do you see in Response B?
2. Rate each response on a scale of 1-10
3. What made the optimized prompt more effective?
"""

comparison = client.models.generate_content(
    model=MODEL_ID,
    contents=comparison_prompt
)

print("=" * 60)
print("FINAL COMPARISON:")
print("=" * 60)
display(Markdown(comparison.text))

## Bonus: Automated optimization loop

Here's a reusable function that combines all steps into a single optimization loop.

In [ ]:
def optimize_prompt(task, initial_prompt, iterations=3, verbose=True):
    """
    Automatically optimize a prompt through iterative self-critique.
    
    Args:
        task: Description of what the prompt should accomplish
        initial_prompt: The starting prompt to optimize
        iterations: Number of critique-rewrite cycles
        verbose: Whether to print intermediate results
    
    Returns:
        Dictionary with optimization history and final results
    """
    history = {
        "prompts": [initial_prompt],
        "responses": [],
        "critiques": []
    }
    
    current_prompt = initial_prompt
    
    for i in range(iterations):
        if verbose:
            print(f"\n{'='*60}")
            print(f"ITERATION {i + 1}")
            print("=" * 60)
        
        # Generate response
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=current_prompt
        )
        history["responses"].append(response.text)
        
        # Critique
        critique = critique_response(task, current_prompt, response.text)
        history["critiques"].append(critique)
        
        if verbose:
            print(f"\nPrompt: {current_prompt[:100]}...")
            print(f"\nCritique summary: {critique[:200]}...")
        
        # Rewrite
        current_prompt = rewrite_prompt(task, current_prompt, critique)
        history["prompts"].append(current_prompt)
    
    # Generate final response with optimized prompt
    final_response = client.models.generate_content(
        model=MODEL_ID,
        contents=current_prompt
    )
    history["responses"].append(final_response.text)
    
    return {
        "initial_prompt": initial_prompt,
        "final_prompt": current_prompt,
        "initial_response": history["responses"][0],
        "final_response": final_response.text,
        "history": history
    }

### Try it with a different task

In [ ]:
# Try optimizing a different weak prompt
result = optimize_prompt(
    task="Write a product description for a fitness tracker",
    initial_prompt="Write about a fitness tracker.",
    iterations=2,
    verbose=True
)

print("\n" + "=" * 60)
print("OPTIMIZATION COMPLETE")
print("=" * 60)
print(f"\nInitial prompt: {result['initial_prompt']}")
print(f"\nFinal prompt: {result['final_prompt']}")

## Key learnings

This self-critique approach reveals common prompt improvements:

1. **Specificity**: Vague prompts get vague responses. The model adds specific requirements.

2. **Structure**: Optimized prompts often request specific formats (bullet points, sections, examples).

3. **Audience**: Defining the target audience helps calibrate complexity and tone.

4. **Constraints**: Adding length limits, focus areas, or exclusions improves relevance.

5. **Context**: Providing background information leads to more informed responses.

You can use this technique to:
- Rapidly iterate on prompts for production applications
- Learn what makes prompts effective for specific tasks
- Generate prompt templates for common use cases
- Debug why certain prompts underperform

## Next steps

### Related prompting techniques

Explore other prompting examples in this repository:

- [Chain of thought prompting](./Chain_of_thought_prompting.ipynb) - Guide the model through reasoning steps
- [Few-shot prompting](./Few_shot_prompting.ipynb) - Provide examples to guide output format
- [Role prompting](./Role_prompting.ipynb) - Assign personas for specialized responses
- [Self-ask prompting](./Self_ask_prompting.ipynb) - Have the model decompose complex questions

### Useful API references

- [Prompt design guide](https://ai.google.dev/gemini-api/docs/prompting-intro)
- [System instructions](https://ai.google.dev/gemini-api/docs/system-instructions)
- [JSON mode for structured outputs](../json_capabilities/)